# Tangelo Velocity: Stage 1 Tutorial

This notebook demonstrates **Tangelo Velocity Stage 1** - a regulatory network model for RNA velocity estimation that integrates chromatin accessibility (ATAC-seq) data.

## Overview

**Stage 1** implements:
- **Sigmoid Feature Transformation**: Learnable smooth feature mapping
- **Linear Interaction Networks**: Gene regulatory interactions with ATAC masking
- **ODE Dynamics**: Cell-specific splicing and degradation modeling
- **Regulatory Loss Functions**: Reconstruction and constraint losses


In [ ]:
# Basic imports (always available)
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Tangelo Velocity
import tangelo_velocity as tv

print(f"Tangelo Velocity version: {tv.__version__}")
print(f"Available modules: {[x for x in tv.__all__ if not x.startswith('_')]}")

In [ ]:
# Check what model components are available
print("=== Model Component Availability ===")

if hasattr(tv, 'models'):
    print("✓ Models module is available")
    
    # List available components
    if hasattr(tv.models, 'list_available_components'):
        available_components = tv.models.list_available_components()
        print(f"Available components ({len(available_components)}):")
        for comp in available_components:
            print(f"  - {comp}")
    
    # Check direct package-level access
    stage1_components = [
        'SigmoidFeatureModule', 'LinearInteractionNetwork', 
        'VelocityODE', 'ODEParameterPredictor',
        'ReconstructionLoss', 'Stage1RegulatoryModel'
    ]
    
    print(f"\nStage 1 component availability:")
    for comp in stage1_components:
        available = hasattr(tv, comp)
        status = "✓" if available else "❌"
        print(f"  {status} {comp}")
        
else:
    print("❌ Models module not available")
    print("This likely means PyTorch dependencies are not installed")
    print("\nYou can still explore:")
    print("  - Configuration system")
    print("  - Synthetic data generation patterns")
    print("  - Stage planning and concepts")

## Stage 1 Configuration

Even without PyTorch, you can explore the configuration system:

In [ ]:
# Stage 1 configuration (always available)
from tangelo_velocity.config import get_stage_config

stage1_config = get_stage_config(stage=1)

print("=== Stage 1 Configuration ===")
print(f"Development Stage: {stage1_config.development_stage}")
print(f"Gene/ATAC dimensions: {stage1_config.gene_dim}/{stage1_config.atac_dim} (inferred from data)")

print("\n=== Regulatory Network Configuration ===")
print(f"Use ATAC masking: {stage1_config.regulatory.use_atac_masking}")
print(f"ATAC threshold: {stage1_config.regulatory.atac_threshold}")
print(f"Sigmoid components: {stage1_config.regulatory.n_sigmoid_components}")
print(f"Interaction strength: {stage1_config.regulatory.interaction_strength}")

print("\n=== ODE Configuration ===")
print(f"Solver: {stage1_config.ode.solver}")
print(f"Time span: {stage1_config.ode.t_span}")
print(f"Beta range: {stage1_config.ode.init_beta_range}")
print(f"Gamma range: {stage1_config.ode.init_gamma_range}")

print("\n=== Training Configuration ===")
print(f"Epochs: {stage1_config.training.n_epochs}")
print(f"Learning rate: {stage1_config.training.learning_rate}")
print(f"Batch size: {stage1_config.training.batch_size}")

## Synthetic Data for Stage 1

Create realistic multi-modal data with regulatory relationships:

In [ ]:
def create_stage1_synthetic_data(n_cells=300, n_genes=100, n_peaks=150, seed=42):
    """Create synthetic data with regulatory structure for Stage 1 demo."""
    
    np.random.seed(seed)
    print(f"Creating synthetic Stage 1 data: {n_cells} cells, {n_genes} genes, {n_peaks} peaks")
    
    # Create cell types with different expression programs
    n_types = 3
    cell_types = np.random.choice(n_types, n_cells)
    
    # Base expression with cell type effects
    base_expr = np.random.gamma(2, 1, (n_cells, n_genes))
    for ct in range(n_types):
        mask = cell_types == ct
        # Each cell type upregulates different gene sets
        gene_start = ct * (n_genes // n_types)
        gene_end = (ct + 1) * (n_genes // n_types)
        base_expr[mask, gene_start:gene_end] *= 2.5
    
    # Simulate splicing dynamics
    splicing_rate = np.random.beta(3, 2, n_genes)  # Gene-specific splicing efficiency
    degradation_rate = np.random.beta(2, 3, n_genes)  # Gene-specific degradation
    
    # Create spliced/unspliced based on kinetics
    unspliced = base_expr * (1 - splicing_rate) + np.random.gamma(0.5, 1, (n_cells, n_genes))
    spliced = base_expr * splicing_rate * (1 - degradation_rate) + np.random.gamma(0.5, 1, (n_cells, n_genes))
    
    # Create regulatory network: peaks -> genes
    peak_gene_links = np.random.binomial(1, 0.25, (n_peaks, n_genes))
    
    # ATAC data correlates with linked gene expression
    atac_data = np.zeros((n_cells, n_peaks))
    for peak in range(n_peaks):
        linked_genes = peak_gene_links[peak, :] > 0
        if linked_genes.sum() > 0:
            # Accessibility correlates with gene activity
            atac_data[:, peak] = spliced[:, linked_genes].mean(axis=1) * 0.3
        atac_data[:, peak] += np.random.gamma(0.3, 1, n_cells)  # Background + noise
    
    # Binary chromatin accessibility for regulatory masking
    accessibility_threshold = np.percentile(atac_data, 75, axis=0)
    open_chromatin = (atac_data > accessibility_threshold).astype(float)
    
    # Gene-level chromatin signal (for regulatory networks)
    gene_chromatin = np.zeros((n_cells, n_genes))
    for gene in range(n_genes):
        linked_peaks = peak_gene_links[:, gene] > 0
        if linked_peaks.sum() > 0:
            gene_chromatin[:, gene] = open_chromatin[:, linked_peaks].mean(axis=1)
        else:
            gene_chromatin[:, gene] = np.random.binomial(1, 0.1, n_cells)  # Background
    
    # Moments for scVelo compatibility
    M_s = spliced + np.random.normal(0, 0.05 * spliced)
    M_u = unspliced + np.random.normal(0, 0.05 * unspliced)
    
    print(f"✓ Created data with {peak_gene_links.sum()} peak-gene regulatory links")
    print(f"✓ {(gene_chromatin > 0.3).sum()} gene-cell accessibility pairs")
    
    return {
        'n_cells': n_cells, 'n_genes': n_genes, 'n_peaks': n_peaks,
        'spliced': spliced.astype(np.float32),
        'unspliced': unspliced.astype(np.float32),
        'M_s': M_s.astype(np.float32),
        'M_u': M_u.astype(np.float32),
        'atac_data': atac_data.astype(np.float32),
        'gene_chromatin': gene_chromatin.astype(np.float32),
        'cell_types': cell_types,
        'peak_gene_links': peak_gene_links,
        'true_splicing_rate': splicing_rate,
        'true_degradation_rate': degradation_rate
    }

# Create demo data
data = create_stage1_synthetic_data(n_cells=300, n_genes=100, n_peaks=150)

In [ ]:
# Visualize the synthetic data structure
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# 1. Spliced vs Unspliced by cell type
for ct in range(3):
    mask = data['cell_types'] == ct
    axes[0, 0].scatter(
        np.log1p(data['spliced'][mask].mean(axis=1)),
        np.log1p(data['unspliced'][mask].mean(axis=1)),
        label=f'Type {ct}', alpha=0.7, s=15
    )
axes[0, 0].set_xlabel('Log(Spliced + 1)')
axes[0, 0].set_ylabel('Log(Unspliced + 1)')
axes[0, 0].set_title('RNA Expression by Cell Type')
axes[0, 0].legend()

# 2. Chromatin accessibility distribution
accessibility_mean = data['gene_chromatin'].mean(axis=0)
axes[0, 1].hist(accessibility_mean, bins=20, alpha=0.7, color='orange')
axes[0, 1].set_xlabel('Mean Gene Accessibility')
axes[0, 1].set_ylabel('Number of Genes')
axes[0, 1].set_title('Chromatin Accessibility Distribution')

# 3. Expression vs Accessibility correlation
expr_mean = data['spliced'].mean(axis=0)
corr = np.corrcoef(expr_mean, accessibility_mean)[0, 1]
axes[0, 2].scatter(expr_mean, accessibility_mean, alpha=0.6, s=15)
axes[0, 2].set_xlabel('Mean Spliced Expression')
axes[0, 2].set_ylabel('Mean Accessibility')
axes[0, 2].set_title(f'Expression vs Accessibility\n(r = {corr:.3f})')

# 4. Splicing efficiency distribution
observed_efficiency = data['spliced'] / (data['spliced'] + data['unspliced'] + 1e-6)
axes[1, 0].hist(observed_efficiency.mean(axis=0), bins=20, alpha=0.7, color='green')
axes[1, 0].set_xlabel('Observed Splicing Efficiency')
axes[1, 0].set_ylabel('Number of Genes')
axes[1, 0].set_title('Splicing Efficiency')

# 5. Peak-gene regulatory network heatmap
network_subset = data['peak_gene_links'][:30, :30]  # Show subset
im = axes[1, 1].imshow(network_subset, cmap='Blues', aspect='auto')
axes[1, 1].set_xlabel('Genes (first 30)')
axes[1, 1].set_ylabel('Peaks (first 30)')
axes[1, 1].set_title('Peak-Gene Regulatory Network')
plt.colorbar(im, ax=axes[1, 1], shrink=0.8)

# 6. Cell type composition
type_counts = np.bincount(data['cell_types'])
axes[1, 2].pie(type_counts, labels=[f'Type {i}' for i in range(len(type_counts))],
               autopct='%1.1f%%', startangle=90)
axes[1, 2].set_title('Cell Type Distribution')

plt.tight_layout()
plt.show()

print("=== Data Summary ===")
print(f"Cells: {data['n_cells']}, Genes: {data['n_genes']}, Peaks: {data['n_peaks']}")
print(f"Regulatory connections: {data['peak_gene_links'].sum()}")
print(f"Mean expression: {data['spliced'].mean():.2f}")
print(f"Mean accessibility: {data['gene_chromatin'].mean():.3f}")
print(f"Expression-accessibility correlation: {corr:.3f}")

## Stage 1 Component Usage (with PyTorch)

If you have PyTorch installed, you can test the individual Stage 1 components:

In [ ]:
# Test Stage 1 components if available
if hasattr(tv, 'models') and hasattr(tv, 'SigmoidFeatureModule'):
    import torch
    
    print("=== Testing Stage 1 Components with PyTorch ===")
    
    # Convert data to tensors
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    spliced_tensor = torch.tensor(data['spliced'], dtype=torch.float32, device=device)
    chromatin_tensor = torch.tensor(data['gene_chromatin'], dtype=torch.float32, device=device)
    
    print(f"Using device: {device}")
    print(f"Data shapes - Spliced: {spliced_tensor.shape}, Chromatin: {chromatin_tensor.shape}")
    
    # 1. Test SigmoidFeatureModule
    sigmoid_module = tv.SigmoidFeatureModule(
        n_genes=data['n_genes'],
        n_components=5,  # Reduced for demo
        init_a=1.0,
        init_b=0.0
    ).to(device)
    
    transformed_features = sigmoid_module(spliced_tensor)
    print(f"\n✓ SigmoidFeatureModule: {spliced_tensor.shape} → {transformed_features.shape}")
    
    # 2. Test LinearInteractionNetwork
    interaction_net = tv.LinearInteractionNetwork(
        n_genes=data['n_genes'],
        use_bias=False,
        interaction_strength=1.0
    ).to(device)
    
    interactions_no_mask = interaction_net(transformed_features)
    interactions_with_mask = interaction_net(transformed_features, atac_mask=chromatin_tensor)
    
    print(f"✓ LinearInteractionNetwork: {transformed_features.shape} → {interactions_with_mask.shape}")
    
    # Compare masking effect
    no_mask_norm = torch.norm(interactions_no_mask, dim=1).mean().item()
    with_mask_norm = torch.norm(interactions_with_mask, dim=1).mean().item()
    print(f"  Interaction strength - No mask: {no_mask_norm:.4f}, With ATAC mask: {with_mask_norm:.4f}")
    print(f"  ATAC masking effect: {with_mask_norm/no_mask_norm:.3f}x")
    
    # 3. Test ODEParameterPredictor  
    param_predictor = tv.ODEParameterPredictor(
        input_dim=data['n_genes'],
        hidden_dim=32,  # Reduced for demo
        init_beta_range=(0.1, 2.0),
        init_gamma_range=(0.1, 1.0)
    ).to(device)
    
    ode_params = param_predictor(interactions_with_mask)
    print(f"\n✓ ODEParameterPredictor: {interactions_with_mask.shape} → {list(ode_params.keys())}")
    
    for param_name, param_tensor in ode_params.items():
        print(f"  {param_name}: shape {param_tensor.shape}, range [{param_tensor.min():.3f}, {param_tensor.max():.3f}]")
    
    # 4. Test Loss Functions
    recon_loss = tv.ReconstructionLoss()
    reg_loss = tv.RegulatoryNetworkLoss()
    
    # Dummy targets for loss computation
    targets = {
        'spliced': spliced_tensor,
        'unspliced': torch.tensor(data['unspliced'], dtype=torch.float32, device=device)
    }
    
    outputs = {
        'pred_spliced': spliced_tensor + torch.randn_like(spliced_tensor) * 0.1,
        'pred_unspliced': targets['unspliced'] + torch.randn_like(targets['unspliced']) * 0.1
    }
    
    recon_loss_val = recon_loss(outputs, targets)
    reg_loss_val = reg_loss(interaction_net.W, chromatin_tensor)
    
    print(f"\n✓ Loss Functions:")
    print(f"  Reconstruction loss: {recon_loss_val.item():.4f}")
    print(f"  Regulatory loss: {reg_loss_val.item():.4f}")
    
    print(f"\n🎉 All Stage 1 components working correctly!")
    
else:
    print("=== Stage 1 Components Not Available ===")
    print("To test components, install required dependencies:")
    print("  pip install torch torchode")
    print("\nOr use Google Colab which has PyTorch pre-installed")
    print("\nEven without PyTorch, you can:")
    print("  ✓ Explore configuration options")
    print("  ✓ Understand data structures")
    print("  ✓ Plan Stage 1 experiments")

## Configuration Customization

Create custom configurations for different research scenarios:

In [ ]:
# Example configurations for different use cases
from tangelo_velocity.config import TangeloConfig

# 1. High regulatory constraint configuration
high_reg_config = TangeloConfig(
    development_stage=1,
    regulatory=tv.config.RegulatoryConfig(
        use_atac_masking=True,
        atac_threshold=0.15,  # Stricter threshold
        n_sigmoid_components=15,  # More components
        interaction_strength=1.5
    ),
    ode=tv.config.ODEConfig(
        solver="dopri5",
        rtol=1e-6,  # Higher precision
        init_beta_range=(0.2, 2.5),
        init_gamma_range=(0.1, 1.2)
    ),
    training=tv.config.TrainingConfig(
        n_epochs=150,
        learning_rate=3e-4,  # Conservative learning rate
        batch_size=256
    )
)

# 2. Fast exploration configuration
fast_config = TangeloConfig(
    development_stage=1,
    regulatory=tv.config.RegulatoryConfig(
        use_atac_masking=False,  # Skip for speed
        n_sigmoid_components=5,   # Minimal components
        interaction_strength=1.0
    ),
    training=tv.config.TrainingConfig(
        n_epochs=50,   # Quick training
        learning_rate=1e-3,
        batch_size=512  # Large batches
    )
)

# 3. Precision configuration for publication
precision_config = TangeloConfig(
    development_stage=1,
    regulatory=tv.config.RegulatoryConfig(
        use_atac_masking=True,
        atac_threshold=0.05,  # Very sensitive
        n_sigmoid_components=25,  # Maximum detail
        interaction_strength=1.2
    ),
    ode=tv.config.ODEConfig(
        solver="dopri5",
        rtol=1e-7,  # Maximum precision
        atol=1e-9,
        max_steps=1500,
        n_time_points=75
    ),
    training=tv.config.TrainingConfig(
        n_epochs=250,
        learning_rate=1e-4,  # Very conservative
        batch_size=128   # Smaller batches for stability
    )
)

configs = {
    "High Regulatory": high_reg_config,
    "Fast Exploration": fast_config, 
    "Publication Precision": precision_config
}

print("=== Stage 1 Configuration Comparison ===")
print(f"{'Configuration':<20} {'ATAC Mask':<10} {'Sigmoid':<8} {'Epochs':<7} {'LR':<8} {'Batch':<6}")
print("-" * 70)

for name, config in configs.items():
    print(f"{name:<20} {str(config.regulatory.use_atac_masking):<10} "
          f"{config.regulatory.n_sigmoid_components:<8} "
          f"{config.training.n_epochs:<7} "
          f"{config.training.learning_rate:<8.0e} "
          f"{config.training.batch_size:<6}")

# Save configurations
for name, config in configs.items():
    filename = f"stage1_{name.lower().replace(' ', '_')}_config.yaml"
    config.save_yaml(filename)
    print(f"✓ Saved {filename}")

print(f"\nLoad configurations with: TangeloConfig.from_yaml('filename.yaml')")

## Summary

This tutorial demonstrated **Tangelo Velocity Stage 1**:

### ✅ What Works Now (Stage 1):
- **Configuration System**: Comprehensive, customizable settings
- **Synthetic Data**: Realistic multi-modal data with regulatory structure  
- **Component Architecture**: Modular design ready for integration
- **PyTorch Components** (if dependencies installed):
  - SigmoidFeatureModule for smooth transformations
  - LinearInteractionNetwork with ATAC masking
  - ODEParameterPredictor for cell-specific dynamics
  - Loss functions for training

### 🚧 Development Roadmap:
- **Stage 0**: ✅ Complete (preprocessing, graphs)
- **Stage 1**: ✅ Complete (regulatory modeling)
- **Stage 2**: 🔄 Planned (graph neural networks)
- **Stage 3**: 🔄 Planned (integrated architecture)
- **Stage 4**: 🔄 Planned (advanced features)

### 🔧 Next Steps:
1. **Install dependencies**: `pip install torch torchode scanpy muon`
2. **Prepare your data**: MuData format with RNA + ATAC modalities
3. **Choose configuration**: Select appropriate Stage 1 settings
4. **Train model**: Use Stage1RegulatoryModel class
5. **Analyze results**: Extract regulatory networks and dynamics

### 📚 Resources:
- [Configuration examples](https://github.com/yourusername/tangelo-velocity/examples/)
- [API documentation](https://tangelo-velocity.readthedocs.io/)
- [Issues & support](https://github.com/yourusername/tangelo-velocity/issues)

**Stage 1 provides a solid foundation for regulatory network-based velocity estimation!**